In [87]:
from pandas import read_csv, DataFrame
import matplotlib.pyplot as plt
import warnings

import pandas as pd
import numpy as np
data_task3 = pd.DataFrame(columns = ['state', 'apple_sold', 'apple_stolen', 'pen_sold', 'pen_stolen'])
data_task3.loc[:,:] = 0

In [88]:
def processing(state, shop):
##################ЧТЕНИЕ ДАННЫХ##################
    data_inv = read_csv('out%2Finput%2F'+state+'-'+shop+'-inventory.csv',
                         ',',
                         index_col=['date'],
                         parse_dates=['date'],
                         dayfirst=True)

    data_sell = read_csv('out%2Finput%2F'+state+'-'+shop+'-sell.csv',
                         ',',
                         index_col=['date'],
                         parse_dates=['date'],
                         dayfirst=True)

    data_supply = read_csv('out%2Finput%2F'+state+'-'+shop+'-supply.csv',
                         ',',
                         index_col=['date'],
                         parse_dates=['date'],
                         dayfirst=True)
    indexes = np.arange(2006, 2016, 1)
    dates = pd.DataFrame(index=pd.date_range(start="2006-01-01", end="2015-12-31"))
    data_task = pd.DataFrame(index = indexes, columns = ['state', 'apple_sold', 'apple_stolen', 'pen_sold', 'pen_stolen'])
    data_task.loc[:,:] = 0
    data_task.loc[:,'state'] = state
##################################################

#################ПОДГОТОВКА ДАННЫХ################    
    data = data_sell.applymap(lambda x: x.find('-pe-', 0, 12))
    data_apple_map = data[data.sku_num == -1]
    data_pen_map = data[data.sku_num == 5]
    data_apple_map = data_apple_map.groupby(['date']).aggregate('count')
    data_pen_map = data_pen_map.groupby(['date']).aggregate('count')
    data_apple = pd.DataFrame(index=pd.date_range(start="2006-01-01", end="2015-12-31"))
    data_pen = pd.DataFrame(index=pd.date_range(start="2006-01-01", end="2015-12-31"))
    data_apple['sku_num'] = data_apple_map['sku_num']
    data_pen['sku_num'] = data_pen_map['sku_num']
    data_apple.fillna(0, inplace=True)
    data_pen.fillna(0, inplace=True)
##################################################

#################РАЗДЕЛЕНИЕ ДАННЫХ################
    data_apple['inv'] = data_inv['apple']
    data_pen['inv'] = data_inv['pen']
    data_apple['sup'] = data_supply['apple']
    data_pen['sup'] = data_supply['pen']
    data_apple.fillna(-1, inplace=True)
    data_pen.fillna(-1, inplace=True)
##################################################

####################ВЫЧИСЛЕНИЕ####################
    data_apple['task1'] = np.zeros(data_apple['sku_num'].size)
    data_pen['task1'] = np.zeros(data_pen['sku_num'].size)
    data_apple['steal'] = np.full(data_apple['sku_num'].size, -1)
    data_pen['steal'] = np.full(data_pen['sku_num'].size, -1)
    temp = 1
    apple_index = data_apple.index.to_datetime();
    for i in range(data_apple.sku_num.size):
        if data_apple['sup'][i] > -1:
            temp += data_apple['sup'][i]
        if data_apple['sku_num'][i] != 'NaN':
            temp -= data_apple['sku_num'][i]
            data_task.loc[apple_index[i].year,'apple_sold'] += data_apple['sku_num'][i]
        if data_apple['inv'][i] > -1:
            data_apple['steal'][i] = temp - data_apple['inv'][i] - 1
            data_task.loc[apple_index[i].year,'apple_stolen'] += data_apple['steal'][i]
            temp = data_apple['inv'][i] + 1
        data_apple['task1'][i] = temp - 1
    temp = 1
    pen_index = data_pen.index.to_datetime();
    for i in range(data_pen.sku_num.size):
        if data_pen['sup'][i] > -1:
            temp += data_pen['sup'][i]
        if data_pen['sku_num'][i] != 'NaN':
            temp -= data_pen['sku_num'][i]
            data_task.loc[pen_index[i].year,'pen_sold'] += data_pen['sku_num'][i]
        if data_pen['inv'][i] > -1:
            data_pen['steal'][i] = temp - data_pen['inv'][i] - 1
            data_task.loc[pen_index[i].year,'pen_stolen'] += data_pen['steal'][i]
            temp = data_pen['inv'][i] + 1
        data_pen['task1'][i] = temp - 1

    #подготовка и создание таблицы для задания 1
    data_task1 = pd.DataFrame(columns=['apple', 'pen'])   
    data_task1['apple'] = data_apple['task1']
    data_task1['pen'] = data_pen['task1']
    data_task1.to_csv('task1'+state+'-'+shop+'.csv', sep=',')    
    
    #подготовка и создание таблицы для задания 2
    data_task2 = pd.DataFrame(columns=['apple', 'pen'])
    data_task2['apple'] = data_apple[data_apple.steal >= 0] ['steal']
    data_task2['pen'] = data_pen[data_pen.steal >= 0] ['steal']
    data_task2.to_csv('task2'+state+'-'+shop+'.csv', sep=',')   
    
    #запись агрегированных данных в сводную таблицу
    global data_task3
    data_task3 = pd.concat([data_task3, data_task])
    ##################################################

In [90]:
processing('MS','b1')
processing('MS','b2')
processing('MS','m1')
processing('MS','m2')
processing('MS','s1')
processing('MS','s2')
processing('MS','s3')
processing('MS','s4')
processing('MS','s5')
data_task3 = data_task3.groupby([data_task3.index, 'state']).aggregate('sum')
print(data_task3)

            apple_sold  apple_stolen  pen_sold  pen_stolen
     state                                                
2006 MS        2152006           418  155633.0         461
2007 MS        2150384           377  154730.0         346
2008 MS        2163559           383  154597.0         382
2009 MS        2152502           433  155409.0         454
2010 MS        2149787           418  155523.0         441
2011 MS        2154860           436  154158.0         452
2012 MS        2160040           381  155798.0         421
2013 MS        2157901           361  154496.0         444
2014 MS        2153434           433  154687.0         441
2015 MS        2152497           370  153562.0         395
